In [2]:
import pandas as pd
import numpy as np

## TODO

1. Assign state labels to each observation (DONE)
2. Sum over observations in each state to calc total counts for each mode in each state from 2006-2017 (DONE)
3. Put these counts into one DataFrame (DONE)
4. Introduce state gas price data, add to df (DONE)
5. Introduce state gas tax data, add to df (DONE)
6. Introduce state demographic controls (still must download), add to df

In [3]:
## prepare transit data to be aggregated by state

transit_df = pd.read_csv('upt.csv', thousands=',')
transit_df = transit_df.fillna(value=0)
transit_df = transit_df[0:2159:]
transit_df = transit_df[transit_df['UZA Name']!='Non-UZA']
transit_df['State'] = transit_df['UZA Name'].map(lambda x: x.split(', ')[1].split('-')[0])
transit_df.loc[transit_df.Agency=='New Jersey Transit Corporation', 'State'] = 'NJ'
transit_df.loc[transit_df.Agency=='Johnson County Kansas, aka: Johnson County Transit', 'State'] = 'KS'
transit_df.loc[transit_df.Agency=='Delaware Transit Corporation','State'] = 'DE'
transit_df.head()

,5 digit NTD ID,4 digit NTD ID,Agency,Active,Reporter Type,UZA,UZA Name,Modes,TOS,JAN02,...,MAR19,APR19,MAY19,JUN19,JUL19,AUG19,SEP19,OCT19,NOV19,State
0,1.0,0001,King County Department of Transportation - Met...,Active,Full Reporter,14.0,"Seattle, WA",DR,PT,135144.0,...,76292.0,78730.0,80718.0,74590.0,79512.0,79704.0,74525.0,83809.0,74197.0,WA
1,1.0,0001,King County Department of Transportation - Met...,Active,Full Reporter,14.0,"Seattle, WA",DT,PT,0.0,...,15343.0,16704.0,16241.0,14544.0,14051.0,13064.0,15666.0,19934.0,15509.0,WA
2,1.0,0001,King County Department of Transportation,Active,Full Reporter,14.0,"Seattle, WA",FB,DO,0.0,...,56148.0,58572.0,69194.0,74257.0,90079.0,67308.0,62058.0,56795.0,34259.0,WA
3,1.0,0001,King County Department of Transportation - Met...,Active,Full Reporter,14.0,"Seattle, WA",LR,DO,12990.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,WA
4,1.0,0001,King County Department of Transportation - Met...,Active,Full Reporter,14.0,"Seattle, WA",MB,DO,6045861.0,...,8602940.0,8890346.0,9188147.0,8607030.0,8824138.0,8696555.0,8398242.0,9546885.0,8444918.0,WA


In [4]:
## format state gas tax data

gtax_df = pd.read_csv('state_gtax.csv')
gtax_df = gtax_df.dropna(how='any')
gtax_df = gtax_df.set_index('STATE')
x = gtax_df.columns[8:]
gtax_df = gtax_df[x]
gtax_df.drop(index='Cali', inplace=True)
gtax_df = .01*gtax_df
gtax_df.head()

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
STATE,,,,,,,,,,,,
Alabama,0.180,0.180,0.180,0.180,0.180,0.180,0.180,0.180,0.180,0.180,0.180,0.180
Alaska,0.080,0.080,0.080,0.080,0.080,0.080,0.080,0.080,0.080,0.080,0.080,0.080
Arizona,0.180,0.180,0.180,0.180,0.180,0.180,0.180,0.180,0.180,0.180,0.180,0.180
Arkansas,0.215,0.215,0.215,0.215,0.215,0.215,0.215,0.215,0.215,0.215,0.215,0.215
California,0.180,0.180,0.180,0.180,0.353,0.357,0.360,0.400,0.400,0.300,0.300,0.420


In [5]:
change_locs = [] 
for i in range(len(gtax_df.index)):
    for j in range(1,len(gtax_df.columns)):
        if gtax_df.iloc[i,j] != gtax_df.iloc[i,j-1]:
            tup = (i,j)
            change_locs.append(tup)
            
change_locs

[(4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 9),
 (4, 11),
 (7, 5),
 (7, 6),
 (8, 3),
 (8, 7),
 (9, 1),
 (9, 2),
 (9, 3),
 (9, 4),
 (9, 5),
 (9, 6),
 (9, 7),
 (10, 7),
 (10, 9),
 (11, 1),
 (11, 10),
 (12, 9),
 (12, 10),
 (14, 11),
 (15, 1),
 (15, 9),
 (15, 10),
 (15, 11),
 (17, 1),
 (17, 2),
 (17, 3),
 (17, 4),
 (17, 5),
 (17, 6),
 (17, 7),
 (17, 9),
 (19, 1),
 (19, 2),
 (19, 3),
 (19, 6),
 (20, 7),
 (20, 9),
 (20, 10),
 (21, 7),
 (22, 11),
 (23, 2),
 (23, 3),
 (23, 4),
 (23, 5),
 (23, 6),
 (24, 7),
 (26, 7),
 (26, 11),
 (27, 1),
 (27, 2),
 (27, 3),
 (27, 4),
 (27, 5),
 (27, 6),
 (27, 7),
 (27, 9),
 (27, 10),
 (27, 11),
 (29, 5),
 (29, 7),
 (29, 9),
 (30, 7),
 (30, 10),
 (30, 11),
 (31, 9),
 (32, 1),
 (32, 2),
 (32, 3),
 (32, 4),
 (32, 5),
 (32, 6),
 (32, 7),
 (32, 9),
 (32, 10),
 (32, 11),
 (33, 1),
 (33, 2),
 (33, 4),
 (33, 5),
 (33, 6),
 (33, 7),
 (33, 9),
 (33, 10),
 (33, 11),
 (37, 5),
 (38, 2),
 (38, 4),
 (38, 7),
 (38, 9),
 (38, 10),
 (38, 11),
 (39, 4),
 (39, 9),
 (40, 11),
 (41, 7

In [6]:
# format gas price data from each state

gprice_df = pd.read_csv('all_state_gprice.csv')
gprice_df = gprice_df.dropna(axis=1, how='any')
gprice_df = gprice_df.set_index('State')
y = gprice_df.columns[36:]
gprice_df = gprice_df[y]
gprice_df = .12033*gprice_df
gprice_df.head()

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
State,,,,,,,,,,,,
AK,2.591908,2.754354,3.590647,2.803689,3.366833,4.144165,4.354743,4.188687,4.052714,3.328328,2.896343,3.227251
AL,2.377721,2.593112,3.104514,2.174363,2.613568,3.312685,3.368037,3.268163,3.115344,2.186396,1.962582,2.226105
AR,2.374111,2.661700,3.075635,2.146687,2.643650,3.352394,3.407746,3.306668,3.152646,2.211665,1.985445,2.252578
AZ,2.490831,2.673733,3.115344,2.254984,2.707425,3.333141,3.502806,3.369240,3.259740,2.677342,2.329589,2.596721
CA,2.583485,2.808502,3.244097,2.475188,2.979371,3.671268,3.879439,3.739856,3.608697,3.064805,2.648463,2.932442


In [7]:
# 1: break up by state and mode
# 2: extract relevant time period (Jan 2006- Dec 2017)
# 3: sum each observation in each column, loop through to get sum for each year
# 4: add data to df with cols: state, year, trips per mode

modes = ['HR', 'LR', 'CR', 'MB', 'TB', 'CB', 'RB']
bus_modes = ['MB', 'TB', 'CB', 'RB']

states = {
        'AK': 'Alaska', 'AL': 'Alabama',
        'AR': 'Arkansas', 'AZ': 'Arizona',
        'CA': 'California', 'CO': 'Colorado',
        'CT': 'Connecticut', 'DC': 'Dist. of Col.',
        'DE': 'Delaware', 'FL': 'Florida',
        'GA': 'Georgia',
        'HI': 'Hawaii', 'IA': 'Iowa',
        'ID': 'Idaho', 'IL': 'Illinois',
        'IN': 'Indiana', 'KS': 'Kansas',
        'KY': 'Kentucky', 'LA': 'Louisiana',
        'MA': 'Massachusetts', 'MD': 'Maryland',
        'ME': 'Maine', 'MI': 'Michigan',
        'MN': 'Minnesota', 'MO': 'Missouri',
        'MS': 'Mississippi', 'MT': 'Montana',
        'NC': 'North Carolina', 'ND': 'North Dakota',
        'NE': 'Nebraska', 'NH': 'New Hampshire',
        'NJ': 'New Jersey', 'NM': 'New Mexico',
        'NV': 'Nevada', 'NY': 'New York',
        'OH': 'Ohio', 'OK': 'Oklahoma',
        'OR': 'Oregon', 'PA': 'Pennsylvania',
        'RI': 'Rhode Island', 'SC': 'South Carolina',
        'SD': 'South Dakota', 'TN': 'Tennessee',
        'TX': 'Texas', 'UT': 'Utah',
        'VA': 'Virginia', 'VT': 'Vermont',
        'WA': 'Washington', 'WI': 'Wisconsin',
        'WV': 'West Virginia', 'WY': 'Wyoming'
}


full_df = pd.DataFrame(columns=['State', 'year', 'gtax', 'gprice', 'HR', 'LR', 'CR', 'Bus'])
years = [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

def get_state_trips(state):
    temp_df = pd.DataFrame(columns=['State', 'year', 'gtax', 'gprice', 'HR', 'LR', 'CR', 'Bus'])
    temp_df['year'] = years
    temp_df['State'] = state
    
    ## get transit data from transit_df (NTD) ##
    for mode in modes:
        state_df = transit_df[transit_df['State']==state]
        xi = state_df.columns.get_loc('JAN06')
        yi = state_df.columns.get_loc('JAN18')
        state_df = state_df.iloc[:,xi:yi]
        state_sum = state_df.sum(axis=0).values
        yr_trips = []
        bus_trips = 12*[0.]

        for i in range(12):
            yr_count = 0
            for j in range(12):
                yr_count += state_sum[12*i+j]
            yr_trips.append(yr_count)
            
        if mode not in bus_modes:
            temp_df[mode] = yr_trips
        else: 
            for i in range(12):
                bus_trips[i] += yr_trips[i]
        
        temp_df['Bus'] = bus_trips
        
    ## get state gas tax data from gtax_df ##
    sname = states[state]
    gtax = gtax_df.loc[sname].values
    temp_df['gtax'] = gtax
    
    ## get state gas price data from gprice_df ## 
    gprice = gprice_df.loc[state].values
    temp_df['gprice'] = gprice
    
    return temp_df

In [8]:
for state in states:
    tdf = get_state_trips(state)
    full_df = full_df.append(tdf)

In [14]:
ints = np.arange(1, 52, 1)
state_inds = dict(zip(states.keys(), ints))
full_df['panelvar'] = 0
for k in state_inds:
    full_df.loc[full_df['State']==k, 'panelvar'] = state_inds[k]

In [17]:
full_df.to_csv('all_state_trips_lag.csv', index=False)

In [15]:
full_df.head()

,State,year,gtax,gprice,HR,LR,CR,Bus,panelvar
0,AK,2006,0.08,2.591908,4600050.0,4600050.0,4600050.0,4600050.0,1
1,AK,2007,0.08,2.754354,4761546.0,4761546.0,4761546.0,4761546.0,1
2,AK,2008,0.08,3.590647,5079974.0,5079974.0,5079974.0,5079974.0,1
3,AK,2009,0.08,2.803689,5078129.0,5078129.0,5078129.0,5078129.0,1
4,AK,2010,0.08,3.366833,5089725.0,5089725.0,5089725.0,5089725.0,1
